# Function calling

https://platform.openai.com/docs/guides/function-calling

<img src="https://cdn.openai.com/API/docs/images/function-calling-diagram-steps.png" alt="Function Calling Diagram" width="600"/>

Function Calling은 OpenAI의 GPT 모델이 특정 작업을 수행할 수 있도록 함수 호출을 지원하는 기능이다.

이 기능을 활용하면 GPT 모델이 단순한 텍스트 생성뿐만 아니라 더 복잡한 작업도 자동으로 처리할 수 있다.

Function Calling을 통해 다음과 같은 작업을 수행할 수 있다:

1. **API 호출**: 외부 API를 호출하여 실시간 데이터를 가져오거나 특정 작업을 수행할 수 있다.  
   예) 날씨 정보 조회, 금융 데이터 조회

2. **데이터 처리**: 특정 함수를 호출하여 데이터 처리 작업을 수행할 수 있다.  
   예) 텍스트 데이터 분석, 통계 계산

3. **자동화된 작업**: 여러 단계의 작업을 자동화하여 수행할 수 있다.  
   예) 사용자가 요청한 내용을 기반으로 보고서 생성, 이메일 작성

**주요 특징**

- **함수 정의**: 사전에 정의된 함수나 API 엔드포인트를 GPT 모델에 통합할 수 있다.
- **자동 호출**: 사용자가 명시적으로 요청하지 않아도, GPT 모델이 컨텍스트를 이해하고 자동으로 적절한 함수를 호출할 수 있다.
- **입출력 처리**: 함수의 입력값을 자동으로 생성하고, 함수 호출 후 반환된 결과를 적절히 처리하여 사용자에게 전달할 수 있다.

## Function 정의

In [ ]:
# 날씨 API : https://openweathermap.org/


from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
OPENWEATHER_API_KEY = userdata.get('OPENWEATHER_API_KEY')

In [ ]:
import requests
import json

def get_current_weather(city, units):
    url = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPENWEATHER_API_KEY}&units={units}'
    response = requests.get(url)
    data = response.json()

    # llm에 전달할 날씨정보

    if response.status_code == 200:
      weather_description = data['weather'][0]['description']
      temperature = data['main']['temp']
      humidity = data['main']['humidity']
      weather_info = {'location': city, 'temperature': temperature, 'humidity': humidity, 'description': weather_description,
                      'unit': 'celsius' if units == 'metric' else 'fahrenheit'}

    else:
      weather_info = {
          'location':city,
          'weather': 'Not Found',
          'temperature': 'Not Found',
          'humidity': 'Not Found',
          'unit': 'Not Found'
      }
    return json.dumps(weather_info)




get_current_weather('서초동', 'metric')

'{"location": "\\uc11c\\ucd08\\ub3d9", "temperature": 24.77, "humidity": 86, "description": "overcast clouds", "unit": "celsius"}'

## LLM 요청 흐름

In [ ]:
from openai import OpenAI

# LLM에게 각 function을 설명하는 메타정보


tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "주어진 도시에 대해 현재 날씨 정보를 가져옵니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "날씨 정보를 얻고자 하는 도시명 혹은 지역명(영문)"
                    },
                    "units": {
                        "type": "string",
                        "enum": ["metric", "imperial"],
                        "description": "섭씨는 metric, 화씨는 imperial"
                    }
                },
                "required": ["city", "units"],
                "additionalProperties": False
            }
        }
    }
]



client = OpenAI(api_key = OPENAI_API_KEY)

messages = [
    {'role': 'system', 'content': '당신은 친절한 챗봇으로, 사용자의 요청에 답변을 제공해주세요.'},
    {'role': 'user', 'content': '서울의 오늘 날씨를 알려줘'}
]

response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages=messages,
    tools=tools
)

print(response)

ChatCompletion(id='chatcmpl-BmXUm4xwdX3ppqYDGcN7ld0Gs4aKE', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_PxYY2WmMM4fVYN472JdJ1kr1', function=Function(arguments='{"city":"Seoul","units":"metric"}', name='get_current_weather'), type='function')]))], created=1750908200, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=20, prompt_tokens=120, total_tokens=140, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [ ]:
# LLM으로부터 응답받은 function_call 정보
response_message = response.choices[0].message
tool_calls = response_message.tool_calls
tool_calls
print(response_message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_PxYY2WmMM4fVYN472JdJ1kr1', function=Function(arguments='{"city":"Seoul","units":"metric"}', name='get_current_weather'), type='function')])


In [ ]:
# 함수목록에서 해당함수를 찾고 실행 - 실행결과를 다시 llm 질의

# 실행가능한 함수목록
available_functions = {
    'get_current_weather': get_current_weather
}

# llm(assistant): function_call 응답
messages.append(response_message) # 'role': 'assistant', 'tool_calls': ...

for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments) # json str -> dict
    # print(function_to_call)
    # print(function_args)
    function_response = function_to_call(**function_args)
    print(function_response)

    messages.append({
        'role': 'tool',
        'tool_call_id': tool_call.id,
        'name': function_name,
        'content': function_response
    })

messages

{"location": "Seoul", "temperature": 23.76, "humidity": 94, "description": "broken clouds", "unit": "celsius"}


[{'role': 'system', 'content': '당신은 친절한 챗봇으로, 사용자의 요청에 답변을 제공해주세요.'},
 {'role': 'user', 'content': '서울의 오늘 날씨를 알려줘'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_PxYY2WmMM4fVYN472JdJ1kr1', function=Function(arguments='{"city":"Seoul","units":"metric"}', name='get_current_weather'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_PxYY2WmMM4fVYN472JdJ1kr1',
  'name': 'get_current_weather',
  'content': '{"location": "Seoul", "temperature": 23.76, "humidity": 94, "description": "broken clouds", "unit": "celsius"}'}]

In [ ]:
# 최종 llm 응답
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages
)

print(response.choices[0].message.content)

오늘 서울의 날씨는 다음과 같습니다:
- **온도**: 23.76도
- **습도**: 94%
- **상태**: 구름 많음

편안한 하루 되세요!


In [1]:
# tool_call을 포함한 함수


def run_conversation(prompt):

  client = OpenAI(api_key = OPENAI_API_KEY)

  messages = [
    {'role': 'system', 'content': '당신은 친절한 챗봇으로, 사용자의 요청에 답변을 제공해주세요.'},
    {'role': 'user', 'content': prompt}
]

  # 첫 번째 LLM 요청
  response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages=messages,
    tools=tools
)
  # 첫 번재 LLM 응답 확인
  response_message = response.choices[0].message
  tool_calls = response_message.tool_calls

  if tool_calls:
    # llm (assistance): function_call 응답
    messages.append(response_message) # 'role': 'assistant', 'tool_calls': ...

    for tool_call in tool_calls:
      function_name = tool_call.function.name
      function_to_call = available_functions[function_name]
      function_args = json.loads(tool_call.function.arguments) # json str -> dict
      function_response = function_to_call(**function_args)

      messages.append({
        'role': 'tool',
        'tool_call_id': tool_call.id,
        'name': function_name,
        'content': function_response
    })

      # 두 번째 LLM 요청

      response = client.chat.completions.create(
      model='gpt-4o-mini',
      messages=messages
)
    return response.choices[0].message.content

  else:
    return response_message.content


print(run_conversation("서울의 날씨를 알려줘"))
print(run_conversation("오늘 밥 먹었니?"))

NameError: name 'OpenAI' is not defined

In [ ]:
import json
from openai import OpenAI

def run_conversation(prompt):
    client = OpenAI(api_key=OPENAI_API_KEY)

    messages = [
        {"role": "system", "content": "당신은 친절한 챗봇입니다."},
        {"role": "user", "content": prompt}
    ]

    # 1차 GPT 호출 → tool_call 요청 유도
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
        messages.append(response_message)  # assistant 메시지(tool_calls 포함)

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_result = function_to_call(**function_args)

            messages.append({
                "role": "function",  # ✅ 공식 role
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": json.dumps(function_result, ensure_ascii=False)  # ✅ 문자열화
            })

        # GPT가 function 결과를 받아 최종 답변 생성
        final_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        return final_response.choices[0].message.content

    else:
        return response_message.content

print(run_conversation("서울의 날씨를 알려줘"))

BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_xKsQHTvXoR2PaVrioifbV5FF", 'type': 'invalid_request_error', 'param': 'messages.[3].role', 'code': None}}